<h1>
    <br style ="font-family:UCL-SoM-Outline;color:#EA7600"> GROUP COURSEWORK </br> 
    </p>
    
    
    
</h1>

<div class="image">

<h4>
          <p style="font-size:18pt">MSIN0166: Data Engineering</p>
              <p style="font-size:18pt">Student Name:</p>

         
  <p>Dominik Wojciechowski</p>        
    <p>Junli Zhu</p>
    <p>Michał Kowalski</p>
    <p>Maxim Mazurov</p>
    <p>lanxin Zhang</p> 
          

</h4>

</div>

<br>
<br>
<br>

# 1. Prepare the problem 




With the popularity of online social media, more and more people are willing to share their lives on social platforms, among them sharing recipes has become a very popular activity.In a wide variety of recipes, how can we infer from the name of the recipe or the key words in the recipe description to know which country it is from? 

But now there are a lot of recipes on the Internet that are not labeled.
If we can tell which country the recipe belongs to by its description and title, we can indirectly infer which country's food is currently the most popular on social media.

For this group assignment, we decide to use different recipes from three countries(India, Italy and Mexico).  We want to do the multi-labeling on those recipes with their correct country. 

First of all, we extract the data we from 'allrecipes' website through scraper and stored them in a dataset.Then, we use pyspark to transfered the dataset into RDD. Thirdly, we do the data cleaning and data preparation for lableling function by using Spark. Finally, we apply label function on those prepared data and get the result.


# 2. Simple Scraper
()

# 3. Convert Dataset to RDD
RDDs are the building blocks of Spark. This part, we will transfer the dataset which we get from the website into RDD.

### PySpark Environment Preparation
Firstly, we bulit a PySpark environment by installing some essential library.

In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pyspark

number_cores = int(os.environ['NUM_CPUS'])
memory_gb = int(os.environ['AVAILABLE_MEMORY_MB']) // 1024
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext(conf=conf)

In [3]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Prepare the Dataset for RDD

Then we load the datassets and labeled those recipes with their countries. Because for this project we will do the multi-label on those recipes so we need do merge those three datasets into one dataset for converting to the RDD.


In [5]:
# Load the dataset
indian = sqlContext.read.csv("/project/Project/DataEngineeringGroupAO/Recipe_dataset/data_indian.csv",header=True)
italian = sqlContext.read.csv("/project/Project/DataEngineeringGroupAO/Recipe_dataset/data_italian.csv",header=True)
mexican = sqlContext.read.csv("/project/Project/DataEngineeringGroupAO/Recipe_dataset/data_mexican.csv",header=True)

In [6]:
# Label the data
from pyspark.sql.functions import lit

indian = indian.withColumn("label",lit("indian"))
italian = italian.withColumn("label",lit("indian"))
mexican = mexican.withColumn("label",lit("mexican"))

In [7]:
indian.show()

+--------------------+--------------------+------+
|               Title|         Description| label|
+--------------------+--------------------+------+
|  Indian Peanut Stew|This is an easy, ...|indian|
|        Roomali Roti|There is no leave...|indian|
|Spicy Sweet Potat...|It's important to...|indian|
|        Chicken Saag|The classic India...|indian|
|Paleo Slow Cooker...|Boneless pork loi...|indian|
|Bombay Chicken an...|Chicken parts are...|indian|
|Indian Carrots, P...|Potatoes, peas an...|indian|
|Wendy's Indian Bu...|This recipe resem...|indian|
|    Indian Chickpeas|Garbanzo beans, o...|indian|
|Dal Makhani (Indi...|These richly spic...|indian|
|               Raita|Chopped tomatoes ...|indian|
|Yogurt-Marinated ...|A yogurt-based ma...|indian|
|Indian-Spiced Roa...|Spicy roasted chi...|indian|
|Cauliflower and T...|Pressed tofu cube...|indian|
|Channa Masala (Ch...|This fantastic In...|indian|
|Bengali Chicken C...|Thy this deliciou...|indian|
|  Indian Sweet Bread|A crisp a

In [8]:
# Combine 3 dataset into one

from functools import reduce
from pyspark.sql import DataFrame

def unionAll(dfs):
    return reduce(DataFrame.unionAll, dfs)

dfs = [indian, italian, mexican]
recipe = unionAll(dfs)
recipe.show()

+--------------------+--------------------+------+
|               Title|         Description| label|
+--------------------+--------------------+------+
|  Indian Peanut Stew|This is an easy, ...|indian|
|        Roomali Roti|There is no leave...|indian|
|Spicy Sweet Potat...|It's important to...|indian|
|        Chicken Saag|The classic India...|indian|
|Paleo Slow Cooker...|Boneless pork loi...|indian|
|Bombay Chicken an...|Chicken parts are...|indian|
|Indian Carrots, P...|Potatoes, peas an...|indian|
|Wendy's Indian Bu...|This recipe resem...|indian|
|    Indian Chickpeas|Garbanzo beans, o...|indian|
|Dal Makhani (Indi...|These richly spic...|indian|
|               Raita|Chopped tomatoes ...|indian|
|Yogurt-Marinated ...|A yogurt-based ma...|indian|
|Indian-Spiced Roa...|Spicy roasted chi...|indian|
|Cauliflower and T...|Pressed tofu cube...|indian|
|Channa Masala (Ch...|This fantastic In...|indian|
|Bengali Chicken C...|Thy this deliciou...|indian|
|  Indian Sweet Bread|A crisp a

In [9]:
recipe.count()

1500

In [15]:
# Convert it to RDD
recipe_rdd = recipe.rdd

In [16]:
recipe_rdd.take(5)

[Row(Title='Indian Peanut Stew', Description='This is an easy, authentic dish from South Asia that appeals to a wide range of tastes. The…', label='indian'),
 Row(Title='Roomali Roti', Description='There is no leavening in this simple, tender Indian flatbread of bread flour, oil, salt and…', label='indian'),
 Row(Title='Spicy Sweet Potato Salad', Description="It's important to use good mayonnaise in this recipe, and to let the cooked potatoes chill…", label='indian'),
 Row(Title='Chicken Saag', Description='The classic Indian chicken and spinach dish gets richness from sour cream.', label='indian'),
 Row(Title='Paleo Slow Cooker Pork Loin', Description='Boneless pork loin slowly cooks in a curried fruit sauce until tender and delicious.', label='indian')]

### note
Not sure if we can split data as what I did in pandas, if not, we could get training, test dataset (csv) prepared before loading them in.

# 3. Data Preperation
### Data Cleaning

We can know that we have 1500 recipes for this project. By looking at the few columns we noticed that there are many useless information contained in our dataset, such as numbers, stop words and punctuation. And alse it mixed the uppercase and the lowercase. For this part, we will remove those noisy information and transfermed all letters into lowercase to get a cleaned dataset for labeling function.   

In [87]:
# Making the RDD Lowercase

recipe_1 = recipe.select(*[lower(col(col_name)).name(col_name) for col_name in recipe.columns])

recipe_rdd_1 = recipe_1.rdd

# Displaying new RDD

recipe_rdd_1.take(5)

[Row(Title='indian peanut stew', Description='this is an easy, authentic dish from south asia that appeals to a wide range of tastes. the…', label='indian'),
 Row(Title='roomali roti', Description='there is no leavening in this simple, tender indian flatbread of bread flour, oil, salt and…', label='indian'),
 Row(Title='spicy sweet potato salad', Description="it's important to use good mayonnaise in this recipe, and to let the cooked potatoes chill…", label='indian'),
 Row(Title='chicken saag', Description='the classic indian chicken and spinach dish gets richness from sour cream.', label='indian'),
 Row(Title='paleo slow cooker pork loin', Description='boneless pork loin slowly cooks in a curried fruit sauce until tender and delicious.', label='indian')]

In [116]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Description", outputCol="Description_1")
recipe_rdd_2 = tokenizer.transform(recipe).select('Title', "Description_1", 'Label')

recipe_rdd_2.printSchema()
recipe_rdd_2.show(10)

root
 |-- Title: string (nullable = true)
 |-- Description_1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Label: string (nullable = false)

+--------------------+--------------------+------+
|               Title|       Description_1| Label|
+--------------------+--------------------+------+
|  Indian Peanut Stew|[this, is, an, ea...|indian|
|        Roomali Roti|[there, is, no, l...|indian|
|Spicy Sweet Potat...|[it's, important,...|indian|
|        Chicken Saag|[the, classic, in...|indian|
|Paleo Slow Cooker...|[boneless, pork, ...|indian|
|Bombay Chicken an...|[chicken, parts, ...|indian|
|Indian Carrots, P...|[potatoes,, peas,...|indian|
|Wendy's Indian Bu...|[this, recipe, re...|indian|
|    Indian Chickpeas|[garbanzo, beans,...|indian|
|Dal Makhani (Indi...|[these, richly, s...|indian|
+--------------------+--------------------+------+
only showing top 10 rows



In [108]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display default list
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [126]:
# Specify input/output columns
remover.setInputCol("Description_1")
remover.setOutputCol("Description_2")

# Transform existing dataframe with the StopWordsRemover
recipe_rdd_3 = remover.transform(recipe_rdd_2).select('Title', "Description_2", 'Label')

# Display
recipe_rdd_3.printSchema()
recipe_rdd_3.show()

root
 |-- Title: string (nullable = true)
 |-- Description_2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Label: string (nullable = false)

+--------------------+--------------------+------+
|               Title|       Description_2| Label|
+--------------------+--------------------+------+
|  Indian Peanut Stew|[easy,, authentic...|indian|
|        Roomali Roti|[leavening, simpl...|indian|
|Spicy Sweet Potat...|[important, use, ...|indian|
|        Chicken Saag|[classic, indian,...|indian|
|Paleo Slow Cooker...|[boneless, pork, ...|indian|
|Bombay Chicken an...|[chicken, parts, ...|indian|
|Indian Carrots, P...|[potatoes,, peas,...|indian|
|Wendy's Indian Bu...|[recipe, resemble...|indian|
|    Indian Chickpeas|[garbanzo, beans,...|indian|
|Dal Makhani (Indi...|[richly, spiced, ...|indian|
|               Raita|[chopped, tomatoe...|indian|
|Yogurt-Marinated ...|[yogurt-based, ma...|indian|
|Indian-Spiced Roa...|[spicy, roasted, ...|indian|
|Cauliflowe

In [127]:
# Removing numbers

from pyspark.sql.functions import when,udf
from pyspark.sql.types import BooleanType
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.types import StructType

def is_digit(value):
    if value:
        return value.isdigit()
    else:
        return False

is_digit_udf = udf(is_digit, BooleanType())

filter_length_udf = udf(lambda row: [x for x in row if not is_digit(x)], ArrayType(StringType()))
recipe_rdd_4 = recipe_rdd_3.withColumn('Description_2', filter_length_udf(col('Description_2')))

In [128]:
recipe_rdd_4 = recipe_rdd_4.rdd
recipe_rdd_4.take(5)

[Row(Title='Indian Peanut Stew', Description_2=['easy,', 'authentic', 'dish', 'south', 'asia', 'appeals', 'wide', 'range', 'tastes.', 'the…'], Label='indian'),
 Row(Title='Roomali Roti', Description_2=['leavening', 'simple,', 'tender', 'indian', 'flatbread', 'bread', 'flour,', 'oil,', 'salt', 'and…'], Label='indian'),
 Row(Title='Spicy Sweet Potato Salad', Description_2=['important', 'use', 'good', 'mayonnaise', 'recipe,', 'let', 'cooked', 'potatoes', 'chill…'], Label='indian'),
 Row(Title='Chicken Saag', Description_2=['classic', 'indian', 'chicken', 'spinach', 'dish', 'gets', 'richness', 'sour', 'cream.'], Label='indian'),
 Row(Title='Paleo Slow Cooker Pork Loin', Description_2=['boneless', 'pork', 'loin', 'slowly', 'cooks', 'curried', 'fruit', 'sauce', 'tender', 'delicious.'], Label='indian')]